In [1]:
import os
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, text
import cx_Oracle
from IPython.core.magic import register_cell_magic
import sys
if sys.version.find('GCC') >= 0:
    domain_name = 'oracle-db'
else:
    domain_name = 'localhost'
    cx_Oracle.init_oracle_client(lib_dir = r"C:\Oracle\instantclient_23_7")
DATABASE_URL = "oracle+cx_oracle://lib:multisqld@{}:1521/?service_name=XEPDB1".format(domain_name)
engine = create_engine(DATABASE_URL)
def get_rows(qry):
    try:
        with engine.connect() as conn:
            result = conn.execute(text(qry))
            df = pd.DataFrame(result.fetchall(), columns = result.keys()).fillna('NULL').pipe(lambda x: x.set_index(x.index + 1))
            return df if len(df) != 1 else df.iloc[0]
    except sqlalchemy.exc.DatabaseError as e:
        orig = e.orig
        offset = getattr(orig, 'offset', None)
        message = getattr(orig, 'message', str(orig))
        print(message, offset)
    except Exception as e:
        print(e.__class__)
        print(e)

def exec_qrys(qrys):
    try:
        results = list()
        with engine.connect() as conn:
            for qry in qrys.split(';'):
                qry = qry.strip()
                if len(qry) == 0:
                    continue
                result = conn.execute(text(qry))
                results.append(
                    "{}, 변경된 행의 수: {}".format(qry, result.rowcount)
                )
        return results
    except Exception as e:
        print(e)

def exec_qry(qry):
    try:
        with engine.connect() as conn:
            result = conn.execute(text(qry))
            conn.commit()
            return "변경된 행의 수: {}".format(result.rowcount)
    except Exception as e:
        print(e)

@register_cell_magic
def SQL(line, cell):
    spt = [i for i in cell.split(';') if len(i.strip()) > 0]
    if len(spt) > 1:
        return exec_qrys(cell)
    sql = cell.strip().replace(';', '')
    if sql.lower().startswith('select'):
        return get_rows(sql)
    else:
        return exec_qry(sql)

# DML(Data Manipulation Language)

|명령|용법|
|----|----|
|INSERT|INSERT INTO 테이블명 [(컬럼1, 컬럼2, … )] <br/> {VALUES (값1, 값2, …)| 서브쿼리};|
|UPDATE|UPDATE 테이블 명 SET<br/>수정할 컬럼1 = 표현1<br/>[, 수정할 컬럼2 = 표현2] …<br/>[WHERE 절]|
|DELETE|DELETE [FROM] 테이블명 [WHERE 절]|
|MERGE|MERGE INTO 타겟 테이블명<br/>USING 소스 테이블명 ON (일치 판별식)<br/>WHEN MATCHED THEN<br/>UPDATE SET<br/>수정할 컬럼1 = 표현1<br/>[, 수정할 컬럼2 = 표현2] …<br/>WHEN NOT MATCHED THEN<br/>INSERT [(컬럼1, 컬럼2, …)]<br/>VALUES (값1, 값2, …)}|

**[예제1]** DML 연습을 해봅니다.

- 연습용 테이블 추가 명령 입니다.

In [2]:
exec_qry("""
CREATE TABLE book_info (
    book_id NUMBER(12),
    title VARCHAR2(200),
    publisher VARCHAR2(100),
    reg_date DATE,
    CONSTRAINT PK_book_info PRIMARY KEY (book_id)
)"""
)

'변경된 행의 수: 0'

In [3]:
%%SQL

SELECT * FROM book_info

,book_id,title,publisher,reg_date


- INSERT를 통해 레코드를 하나 추가합니다.

  INSERT INTO 테이블명(컬럼명|[컬럼명2], ...) VALUES (....)

In [4]:
%%SQL

INSERT INTO book_info(book_id, title, publisher, reg_date)
VALUES (1, '해리포터', '외국출판사', to_date('2025-05-07', 'yyyy-mm-dd'))

'변경된 행의 수: 1'

In [5]:
%%SQL

SELECT * FROM book_info

book_id                        1
title                       해리포터
publisher                  외국출판사
reg_date     2025-05-07 00:00:00
Name: 1, dtype: object

- 하나 더 추가합니다.

In [6]:
%%SQL

INSERT INTO book_info
VALUES (1, '반지의 제왕', '보석출판사', to_date('2025_06-01', 'yyyy-mm-dd'))

(cx_Oracle.IntegrityError) ORA-00001: unique constraint (LIB.PK_BOOK_INFO) violated
[SQL: INSERT INTO book_info
VALUES (1, '반지의 제왕', '보석출판사', to_date('2025_06-01', 'yyyy-mm-dd'))]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


인덱스가 중복하기 때문에 추가가 되지 않았습니다.

- 컬럼을 지정하지 않으면 등록된 컬럼 순에 맞춰 입력이 됩니다. 이렇게 쓰면 부작용이 있어 쓰지 않습니다.  개념과 외부 스키마 간에 독립성이 생깁니다.

In [7]:
%%SQL

INSERT INTO book_info
VALUES (2, '반지의 제왕', '보석출판사', to_date('2025_06-01', 'yyyy-mm-dd'))

'변경된 행의 수: 1'

In [8]:
%%SQL

SELECT * FROM book_info

,book_id,title,publisher,reg_date
1,1,해리포터,외국출판사,2025-05-07
2,2,반지의 제왕,보석출판사,2025-06-01


- UPDATE 를 합니다. WHERE 조건에 해당하는 행들을 바꿉니다.

In [9]:
%%SQL

UPDATE book_info SET publisher = '영국출판사' WHERE book_id = 1

'변경된 행의 수: 1'

In [10]:
%%SQL

SELECT * FROM book_info

,book_id,title,publisher,reg_date
1,1,해리포터,영국출판사,2025-05-07
2,2,반지의 제왕,보석출판사,2025-06-01


In [11]:
%%SQL

UPDATE book_info SET reg_date = reg_date - 1

'변경된 행의 수: 2'

In [12]:
%%SQL

SELECT * FROM book_info

,book_id,title,publisher,reg_date
1,1,해리포터,영국출판사,2025-05-06
2,2,반지의 제왕,보석출판사,2025-05-31


- DELETE FROM 테이블에서 조건해당하는 행을 삭제합니다.


In [13]:
%%SQL

DELETE FROM book_info WHERE book_id = (SELECT max(book_id) FROM book_info)

'변경된 행의 수: 1'

In [14]:
%%SQL

SELECT * FROM book_info

book_id                        1
title                       해리포터
publisher                  영국출판사
reg_date     2025-05-06 00:00:00
Name: 1, dtype: object

In [15]:
%%SQL

SELECT * FROM book1

,book_id,book_title,book_categ_id,publisher,pub_date
1,2219,사피엔스,1062,김영사,2015-11-01
2,1097,문학이란 무엇인가,509,민음사,1998-09-01
3,1061,양철북 2,489,민음사,2000-03-01
4,3270,"뿌리깊은 초등국어 독해력 어휘편 3단계 (초등3,4학년)",1550,마더텅,2019-12-01
5,3233,"뿌리깊은 초등국어 독해력 1단계 (초등1,2학년)",1535,마더텅,2018-03-01
6,2497,생명이란 무엇인가,1196,까치(까치글방),2021-01-01
7,1137,잃어버린 아이 이야기,534,한길사,2017-12-01
8,1059,데미안,489,민음사,2000-12-01
9,3345,초등영어 받아쓰기·듣기 10회 모의고사 초등 6학년 1,1568,마더텅,2019-04-01
10,1092,동주와 빈센트,503,저녁달고양이,2019-09-01


In [16]:
%%SQL 

SELECT book_id, book_title, publisher, sysdate 
FROM book1
WHERE publisher like '%사'

,book_id,book_title,publisher,sysdate
1,2219,사피엔스,김영사,2025-08-10 09:41:03
2,1097,문학이란 무엇인가,민음사,2025-08-10 09:41:03
3,1061,양철북 2,민음사,2025-08-10 09:41:03
4,1137,잃어버린 아이 이야기,한길사,2025-08-10 09:41:03
5,1059,데미안,민음사,2025-08-10 09:41:03


- SELECT 쿼리에서 반환된 행들을 입력합니다.

In [17]:
%%SQL

INSERT INTO book_info(book_id, title, publisher, reg_date)
SELECT book_id, book_title, publisher, sysdate 
FROM book1
WHERE publisher like '%사'

'변경된 행의 수: 5'

In [18]:
%%SQL

SELECT * FROM book_info

,book_id,title,publisher,reg_date
1,1,해리포터,영국출판사,2025-05-06 00:00:00
2,1059,데미안,민음사,2025-08-10 09:41:03
3,1061,양철북 2,민음사,2025-08-10 09:41:03
4,1097,문학이란 무엇인가,민음사,2025-08-10 09:41:03
5,1137,잃어버린 아이 이야기,한길사,2025-08-10 09:41:03
6,2219,사피엔스,김영사,2025-08-10 09:41:03


In [19]:
%%SQL

UPDATE book_info SET title = '', publisher = publisher || '~~~' WHERE length(publisher) = 3

'변경된 행의 수: 5'

In [20]:
%%SQL

SELECT * FROM book_info

,book_id,title,publisher,reg_date
1,1,해리포터,영국출판사,2025-05-06 00:00:00
2,1059,NULL,민음사~~~,2025-08-10 09:41:03
3,1061,NULL,민음사~~~,2025-08-10 09:41:03
4,1097,NULL,민음사~~~,2025-08-10 09:41:03
5,1137,NULL,한길사~~~,2025-08-10 09:41:03
6,2219,NULL,김영사~~~,2025-08-10 09:41:03


In [21]:
%%SQL

SELECT * FROM book1

,book_id,book_title,book_categ_id,publisher,pub_date
1,2219,사피엔스,1062,김영사,2015-11-01
2,1097,문학이란 무엇인가,509,민음사,1998-09-01
3,1061,양철북 2,489,민음사,2000-03-01
4,3270,"뿌리깊은 초등국어 독해력 어휘편 3단계 (초등3,4학년)",1550,마더텅,2019-12-01
5,3233,"뿌리깊은 초등국어 독해력 1단계 (초등1,2학년)",1535,마더텅,2018-03-01
6,2497,생명이란 무엇인가,1196,까치(까치글방),2021-01-01
7,1137,잃어버린 아이 이야기,534,한길사,2017-12-01
8,1059,데미안,489,민음사,2000-12-01
9,3345,초등영어 받아쓰기·듣기 10회 모의고사 초등 6학년 1,1568,마더텅,2019-04-01
10,1092,동주와 빈센트,503,저녁달고양이,2019-09-01


- MERGE INTO  구분을 사용해봅니다.

In [22]:
%%SQL

MERGE INTO book_info a 
USING (SELECT book_id, book_title, publisher, pub_date FROM book1) b ON (a.book_id = b.book_id)
WHEN MATCHED THEN
UPDATE SET 
    title = b.book_title,  publisher = b.publisher
WHEN NOT MATCHED THEN
INSERT (book_id, title, publisher, reg_date)
VALUES (b.book_id, b.book_title, b.publisher, sysdate)

'변경된 행의 수: 10'

In [23]:
%%SQL

SELECT * FROM book_info

,book_id,title,publisher,reg_date
1,1,해리포터,영국출판사,2025-05-06 00:00:00
2,1092,동주와 빈센트,저녁달고양이,2025-08-10 09:41:03
3,1059,데미안,민음사,2025-08-10 09:41:03
4,1061,양철북 2,민음사,2025-08-10 09:41:03
5,1097,문학이란 무엇인가,민음사,2025-08-10 09:41:03
6,1137,잃어버린 아이 이야기,한길사,2025-08-10 09:41:03
7,2219,사피엔스,김영사,2025-08-10 09:41:03
8,2497,생명이란 무엇인가,까치(까치글방),2025-08-10 09:41:03
9,3233,"뿌리깊은 초등국어 독해력 1단계 (초등1,2학년)",마더텅,2025-08-10 09:41:03
10,3270,"뿌리깊은 초등국어 독해력 어휘편 3단계 (초등3,4학년)",마더텅,2025-08-10 09:41:03


# TCL(Transaction Control Language)

|명령|설명|
|-----|----|
|COMMIT|변경 사항을 반영<br/>변경 이전 데이터 기억 해제<br/>모든 사용자가 변경 사항 조회 가능<br/>관련 행에 대한 잠금 해제. 타 사용자 조작가능|
|ROLLBACK<br/>\[포인트명\]|변경 사항 취소<br/>관련 행에 대한 잠금 해제. 타 사용자 조작가능<br/>포인트명을 지정하면, SAVEPOINT가 지정된 이후의 변경 사항 취소<br/>포인트명 미지정시, 모두 취소|
|SAVEPOINT<br/>포인트명|트랜잭션 내에서 ROLLBACK 시 복귀 지점 설정|


**[예제 2]** 트랜젝션 기능을 확인합니다.

- TRUNCATE 테이블은 Transaction에 포함되지 않고, 즉각 삭제 작업이 반영됩니다.

In [24]:
%%SQL

TRUNCATE table book_info;
ROLLBACK;

['TRUNCATE table book_info, 변경된 행의 수: 0', 'ROLLBACK, 변경된 행의 수: 0']

In [25]:
%%SQL

SELECT * FROM book_info

,book_id,title,publisher,reg_date


In [26]:
%%SQL

INSERT INTO book_info(book_id, title, publisher, reg_date)
VALUES (1, '해리포터', '외국출판사', to_date('2025-05-07', 'yyyy-mm-dd'));

INSERT INTO book_info
VALUES (2, '반지의 제왕', '보석출판사', to_date('2025_06-01', 'yyyy-mm-dd'));

SAVEPOINT sp1;

INSERT INTO book_info (book_id, title, publisher, reg_date)
VALUES (3, '명탐정 코난', '일본출판사', SYSDATE);

ROLLBACK TO sp1;

COMMIT;

["INSERT INTO book_info(book_id, title, publisher, reg_date)\nVALUES (1, '해리포터', '외국출판사', to_date('2025-05-07', 'yyyy-mm-dd')), 변경된 행의 수: 1",
 "INSERT INTO book_info\nVALUES (2, '반지의 제왕', '보석출판사', to_date('2025_06-01', 'yyyy-mm-dd')), 변경된 행의 수: 1",
 'SAVEPOINT sp1, 변경된 행의 수: 0',
 "INSERT INTO book_info (book_id, title, publisher, reg_date)\nVALUES (3, '명탐정 코난', '일본출판사', SYSDATE), 변경된 행의 수: 1",
 'ROLLBACK TO sp1, 변경된 행의 수: 0',
 'COMMIT, 변경된 행의 수: 0']

In [27]:
%%SQL

SELECT * FROM book_info

,book_id,title,publisher,reg_date
1,1,해리포터,외국출판사,2025-05-07
2,2,반지의 제왕,보석출판사,2025-06-01


- 비정상 종료시 롤백됩니다.

- 접속해제시 커밋됩니다.

In [28]:
%%SQL

INSERT INTO book_info(book_id, title, publisher, reg_date)
VALUES (4, '해리포터', '외국출판사', to_date('2025-05-07', 'yyyy-mm-dd'));

INSERT INTO book_info
VALUES (5, '반지의 제왕', '보석출판사', to_date('2025_06-01', 'yyyy-mm-dd'));

SAVEPOINT sp1;

INSERT INTO book_info (book_id, title, publisher, reg_date)
VALUES (4, '명탐정 코난', '일본출판사', SYSDATE);

ROLLBACK TO sp1;

COMMIT;

(cx_Oracle.IntegrityError) ORA-00001: unique constraint (LIB.PK_BOOK_INFO) violated
[SQL: INSERT INTO book_info (book_id, title, publisher, reg_date)
VALUES (4, '명탐정 코난', '일본출판사', SYSDATE)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [29]:
%%SQL

SELECT * FROM book_info

,book_id,title,publisher,reg_date
1,1,해리포터,외국출판사,2025-05-07
2,2,반지의 제왕,보석출판사,2025-06-01


# DDL(Data Definition Language)

## CREATE TABLE

```SQL
CREATE TABLE 테이블명(
   컬럼명1   데이터유형 [기본값1] [NOT NULL]
   ,컬럼명2  데이터유형 [기본값2] [NOT NULL]
   ,…
   [, CONSTRAINT 제약명1 제약 조건1,  …]
);
```

**제약조건**

|종류|설명|
|----|-----|
|PRIMARY KEY|식별자로 사용하는 기본키 지정<br/>테이블 당 하나만 정의 가능<br/>UNIQUE와 NOT NULL 제약을 포함|
|UNIQUE|고유값 제약<br/>NULL은 제약 대상에서 빠진다|
|NOT NULL|NULL 값 미허용|
|CHECK|입력할 수 있는 값에 대한 제약|
|FOREIGN KEY|외래키 지정|


**명명 규칙**

<table>
  <tr>
    <th >테이블</th><th>컬럼</th>
  </tr>
  <tr>
    <td>객체 의미를 나타낼 수 있는 이름<br/>가능한 단수<br/>데이터베이스에서 중복불가</td>
    <td>테이블 내에서 중복 불가<br/>일관성 있게 사용하는 것을 권장</td>
  </tr>
  <tr>
    <td colspan="2">반드시 문자로 시작<br/>예약어 사용불가<br/>허용 글자: A-Z, a-z, 0-9, %, #<br/>대소문자를 구분하지 않지만, 내부적으로 대문자로 만들어짐</td>
  </tr>
</table>

```SQL
CREATE TABLE 테이블명 AS SELECT 구문
```


**[예제 3]** DDL의 기능을 확인해봅니다.

- book_stock 테이블을 생성합니다.

 CHECK 는 도메인의 조건을 나타냅니다.

In [30]:
%%SQL

CREATE TABLE book_stock (
    book_id NUMBER(12) NOT NULL,
    stock_seq NUMBER(10) CHECK(stock_seq > 0),
    book_status NUMBER(8) CHECK (book_status in (1, 2, 3, 4)),
    book_location VARCHAR(100) UNIQUE, 
    reg_date DATE,
    CONSTRAINT PK_BOOK_STOCK PRIMARY KEY (book_id, stock_seq),
    CONSTRAINT FK_BOOK_STOCK FOREIGN KEY (book_id) REFERENCES book(book_id) ON DELETE CASCADE
);

'변경된 행의 수: 0'

In [31]:
%%SQL

SELECT * FROM book_stock

,book_id,stock_seq,book_status,book_location,reg_date


In [32]:
%%SQL

SELECT * 
FROM book
FETCH FIRST 5 ROWS ONLY

,book_id,book_title,book_subtitle,book_categ_id,publisher,authors,summary,pub_date,reg_date
1,2924,디렉터 6과 링고,NULL,1409,대림,이제민,"디렉터의 기초부터 다양한 애니메이션을 만드는 기법, 그리고 디렉터 언어인 링고 스크...",1999-02-01,1999-02-01 12:40:53
2,2231,마르크스 그 가능성의 중심,NULL,1070,이산,가라타니 고진|김경원,"이 책은 마르크스 '자본론'의 가치형태론을 구조주의적 언어분석을 통해 읽고, 일본문...",1999-05-01,1999-05-01 11:18:17
3,2008,도예 기초 실기,NULL,953,예경,한홍곤,처음 도예를 배우는 사람에게 도움을 주기 위해 저자가 30년 가까이 교육 현장에서 ...,1999-08-01,1999-08-01 13:44:46
4,517,제제공학,NULL,250,신일상사,제제공학편찬위원회,NULL,1999-09-01,1999-09-01 15:51:15
5,2443,한국 공동주택계획의 역사,NULL,1174,세진사,강부성,NULL,1999-10-01,1999-10-01 12:56:03


- 레코드를 하나 넣어 봅니다.

In [33]:
%%SQL

INSERT INTO book_stock(book_id, stock_seq, book_status, book_location, reg_date)
VALUES (2924, 0, 1, '1 도서관', sysdate)

(cx_Oracle.IntegrityError) ORA-02290: check constraint (LIB.SYS_C008417) violated
[SQL: INSERT INTO book_stock(book_id, stock_seq, book_status, book_location, reg_date)
VALUES (2924, 0, 1, '1 도서관', sysdate)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


stock_seq > 0 인 도메인 조건에 어긋나 에러가 납니다.

In [34]:
%%SQL

INSERT INTO book_stock(book_id, stock_seq, book_status, book_location, reg_date)
VALUES (2924, 1, 1, '1 도서관', sysdate)

'변경된 행의 수: 1'

In [35]:
%%SQL

SELECT max(stock_seq) + 1 FROM book_stock WHERE book_id = 2924

MAX(STOCK_SEQ)+1    2
Name: 1, dtype: int64

- book_location에는 UNIQUE 제약이 있습니다. 위와 동일한 book_location을 넣어 봅니다.

In [36]:
%%SQL

INSERT INTO book_stock(book_id, stock_seq, book_status, book_location, reg_date)
VALUES
    (2924, (SELECT max(stock_seq) + 1 FROM book_stock WHERE book_id = 2924), 1, '1 도서관', sysdate)

(cx_Oracle.IntegrityError) ORA-00001: unique constraint (LIB.SYS_C008420) violated
[SQL: INSERT INTO book_stock(book_id, stock_seq, book_status, book_location, reg_date)
VALUES
    (2924, (SELECT max(stock_seq) + 1 FROM book_stock WHERE book_id = 2924), 1, '1 도서관', sysdate)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


- UNIQUE 제약이 어긋나 에러가 납니다.

In [37]:
%%SQL

INSERT INTO book_stock(book_id, stock_seq, book_status, book_location, reg_date)
VALUES
    (2924, (SELECT max(stock_seq) + 1 FROM book_stock WHERE book_id = 2924), 1, '2 도서관', sysdate)

'변경된 행의 수: 1'

In [38]:
%%SQL

INSERT INTO book_stock(book_id, stock_seq, book_status, book_location, reg_date)
VALUES
    (2231, (SELECT nvl(max(stock_seq), 0) + 1 FROM book_stock WHERE book_id = 2231), 1, '3 도서관', sysdate)

'변경된 행의 수: 1'

In [39]:
%%SQL

SELECT a.*, b.book_title, b.publisher
FROM book_stock a LEFT OUTER JOIN book b ON a.book_id =  b.book_id

,book_id,stock_seq,book_status,book_location,reg_date,book_title,publisher
1,2924,1,1,1 도서관,2025-08-10 09:41:03,디렉터 6과 링고,대림
2,2924,2,1,2 도서관,2025-08-10 09:41:03,디렉터 6과 링고,대림
3,2231,1,1,3 도서관,2025-08-10 09:41:03,마르크스 그 가능성의 중심,이산


## ALTER TABLE

|명령|설명|
|-----|----|
|ADD |ALTER TABLE 테이블명<br/>ADD (추가할 컬럼명1 데이터유형 [기본 값] [NOT NULL]<br/>[, 추가할 컬럼명1 데이터유형 [기본 값] [NOT NULL]<br/>, …]);|
|DROP COLUMN|ALTER TABLE 테이블명 DROP (삭제할 컬럼1[, 삭제할 컬럼2, …]);<br/>MODIFY COLUMN	ALTER TABLE 테이블명<br/>MODIFY (컬럼명1 테이블유형1 [기본 값1] [NOT NULL]<br/>[, 컬럼명2, 테이블유형2 [기본 값1] [NOT NULL]<br/>, …]);|
|RENAME COLUMN|ALTER TABLE 테이블명 RENAME COLUMN 기존 컬럼명 TO  새로운 컬럼명|
|DROP CONSTRAINT|ALTER TABLE 테이블명 DROP CONSTRAINT 제약조건명;|
|ADD CONSTRAINT|ALTER TABLE 테이블명 ADD CONSTRAINT 제약조건명 제약조건;	|



**[예제 4]** ALTER TABLE의 기능을 알아 봅니다.

In [40]:
%%SQL

SELECT * FROM book_stock

,book_id,stock_seq,book_status,book_location,reg_date
1,2924,1,1,1 도서관,2025-08-10 09:41:03
2,2924,2,1,2 도서관,2025-08-10 09:41:03
3,2231,1,1,3 도서관,2025-08-10 09:41:03


- book_manager 컬럼을 추가합니다.

In [41]:
%%SQL

ALTER TABLE book_stock
ADD book_manager VARCHAR2(100)

'변경된 행의 수: 0'

In [42]:
%%SQL

SELECT * FROM book_stock

,book_id,stock_seq,book_status,book_location,reg_date,book_manager
1,2924,1,1,1 도서관,2025-08-10 09:41:03,NULL
2,2924,2,1,2 도서관,2025-08-10 09:41:03,NULL
3,2231,1,1,3 도서관,2025-08-10 09:41:03,NULL


- book_manager 컬럼을 book_man으로 바꿉니다.

In [43]:
%%SQL

ALTER TABLE book_stock
RENAME COLUMN book_manager TO book_man

'변경된 행의 수: 0'

In [44]:
%%SQL 

SELECT * FROM book_stock

,book_id,stock_seq,book_status,book_location,reg_date,book_man
1,2924,1,1,1 도서관,2025-08-10 09:41:03,NULL
2,2924,2,1,2 도서관,2025-08-10 09:41:03,NULL
3,2231,1,1,3 도서관,2025-08-10 09:41:03,NULL


- book_man 컬럼에 제약을 추가합니다.

In [45]:
%%SQL

ALTER TABLE book_stock
ADD CONSTRAINT book_stock_book_man_unique UNIQUE(book_man)

'변경된 행의 수: 0'

In [46]:
%%SQL

UPDATE book_stock SET book_man = '강선구';

(cx_Oracle.IntegrityError) ORA-00001: unique constraint (LIB.BOOK_STOCK_BOOK_MAN_UNIQUE) violated
[SQL: UPDATE book_stock SET book_man = '강선구']
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [47]:
%%SQL

UPDATE book_stock SET book_man = '강선구' || to_char(ROWNUM)

'변경된 행의 수: 3'

In [48]:
%%SQL

SELECT * FROM book_stock

,book_id,stock_seq,book_status,book_location,reg_date,book_man
1,2924,1,1,1 도서관,2025-08-10 09:41:03,강선구1
2,2924,2,1,2 도서관,2025-08-10 09:41:03,강선구2
3,2231,1,1,3 도서관,2025-08-10 09:41:03,강선구3


- 제약을 제거합니다.

In [49]:
%%SQL 

ALTER TABLE book_stock
DROP CONSTRAINT book_stock_book_man_unique

'변경된 행의 수: 0'

In [50]:
%%SQL

UPDATE book_stock SET book_man = '강선구' 

'변경된 행의 수: 3'

In [51]:
%%SQL

SELECT * FROM book_stock

,book_id,stock_seq,book_status,book_location,reg_date,book_man
1,2924,1,1,1 도서관,2025-08-10 09:41:03,강선구
2,2924,2,1,2 도서관,2025-08-10 09:41:03,강선구
3,2231,1,1,3 도서관,2025-08-10 09:41:03,강선구


- 컬럼을 제거합니다.

In [52]:
%%SQL

ALTER TABLE book_stock
DROP COLUMN book_man

'변경된 행의 수: 0'

In [53]:
%%SQL

SELECT * FROM book_stock

,book_id,stock_seq,book_status,book_location,reg_date
1,2924,1,1,1 도서관,2025-08-10 09:41:03
2,2924,2,1,2 도서관,2025-08-10 09:41:03
3,2231,1,1,3 도서관,2025-08-10 09:41:03


## 기타 DDL

|명령|설명|
|----|----|
|ALTER TABLE ~ RENAME TO ~|ALTER TABLE 기존 테이블명 RENAME TO 새로운 테이블명;<br/>sp_rename ‘기존 테이블명’, ‘새로운 테이블명‘;|
|DROP TABLE|DROP TABLE 테이블명 [CASCADE CONSTRAINT];|
|TRUNCATE TABLE|TRUNCATE TABLE 테이블명; <br/> 테이블은 사라지지 않고, 테이블 내용이 초기화됩니다. <br/>Transaction에 영향을 받지 않습니다.|


**[예제 5]** ALTER TABLE ~ RENAME TO, DROP TABLE, TRUNCATE TABLE을 알아봅니다.

In [54]:
%%SQL

ALTER TABLE book_stock RENAME TO book_stock_info

'변경된 행의 수: 0'

In [55]:
%%SQL

SELECT * FROM book_stock_info

,book_id,stock_seq,book_status,book_location,reg_date
1,2924,1,1,1 도서관,2025-08-10 09:41:03
2,2924,2,1,2 도서관,2025-08-10 09:41:03
3,2231,1,1,3 도서관,2025-08-10 09:41:03


In [56]:
%%SQL

TRUNCATE TABLE book_stock_info

'변경된 행의 수: 0'

In [57]:
%%SQL

DROP TABLE book_stock_info

'변경된 행의 수: 0'

# DCL

|기능|명령|
|----|----|
|권한 부여|GRANT 권한 TO {사용자ID|ROLE};|
|권한 회수|REVOKE 권한 FROM {사용자ID|ROLE};|
|역할 Role 생성|CREATE ROLE 역할명;|
|역할(Role) 부여|GRANT 역할명 TO 사용자ID;|
|역할(Role) 회수|REVOKE 역할명 FROM 사용자ID;|


- DBA 계정으로 접근을 하고 사용자를 추가 합니다.

In [58]:
DATABASE_URL = "oracle+cx_oracle://sys:multisqld@{}:1521/xe?mode=SYSDBA".format(domain_name)
engine = create_engine(DATABASE_URL)

In [59]:
%%SQL
ALTER SESSION SET CONTAINER = XEPDB1;

'변경된 행의 수: 0'

- test / test123 인 사용자를 추가합니다.

In [60]:
%%SQL

CREATE USER TEST IDENTIFIED BY test123;

'변경된 행의 수: 0'

In [61]:
%%SQL

ALTER USER TEST DEFAULT TABLESPACE LIB_SPACE

'변경된 행의 수: 0'

- 추가한 test / test123으로 접근합니다.

In [62]:
DATABASE_URL = "oracle+cx_oracle://test:test123@{}:1521/?service_name=XEPDB1".format(domain_name)
engine = create_engine(DATABASE_URL)

In [63]:
%%SQL

SELECT * FROM book1

ORA-01045: user TEST lacks CREATE SESSION privilege; logon denied None


SESSION을 만들 권한이 없어 logon 이 거부됩니다.

In [64]:
DATABASE_URL = "oracle+cx_oracle://sys:multisqld@{}:1521/xe?mode=SYSDBA".format(domain_name)
engine = create_engine(DATABASE_URL)

In [65]:
%%SQL
ALTER SESSION SET CONTAINER = XEPDB1;

'변경된 행의 수: 0'

- 접근 권한을 부여합니다.

In [66]:
%%SQL 

GRANT CONNECT TO TEST

'변경된 행의 수: 0'

- SESSION 생성 권한들 부여합니다.

In [67]:
%%SQL

GRANT CREATE SESSION TO TEST

'변경된 행의 수: 0'

- 테이블 공간에 대한 무한 접근 권한을 부여합니다.

In [68]:
%%SQL

GRANT UNLIMITED TABLESPACE TO TEST

'변경된 행의 수: 0'

In [69]:
DATABASE_URL = "oracle+cx_oracle://test:test123@{}:1521/?service_name=XEPDB1".format(domain_name)
engine = create_engine(DATABASE_URL)

In [70]:
%%SQL

SELECT * FROM tab

,tname,tabtype,clusterid


권한을 받으면 정상 접근이 됩니다.

In [71]:
DATABASE_URL = "oracle+cx_oracle://sys:multisqld@{}:1521/xe?mode=SYSDBA".format(domain_name)
engine = create_engine(DATABASE_URL)

In [72]:
%%SQL
ALTER SESSION SET CONTAINER = XEPDB1;

'변경된 행의 수: 0'

- TEST 유저에 lib.book1에 SELECT 권한을 부여합니다.

In [73]:
%%SQL

GRANT SELECT ON lib.book1 TO TEST;

'변경된 행의 수: 0'

In [74]:
DATABASE_URL = "oracle+cx_oracle://test:test123@{}:1521/?service_name=XEPDB1".format(domain_name)
engine = create_engine(DATABASE_URL)

- book1의 내용을 조회합니다.

In [75]:
%%SQL

SELECT * FROM lib.book1

,book_id,book_title,book_categ_id,publisher,pub_date
1,2219,사피엔스,1062,김영사,2015-11-01
2,1097,문학이란 무엇인가,509,민음사,1998-09-01
3,1061,양철북 2,489,민음사,2000-03-01
4,3270,"뿌리깊은 초등국어 독해력 어휘편 3단계 (초등3,4학년)",1550,마더텅,2019-12-01
5,3233,"뿌리깊은 초등국어 독해력 1단계 (초등1,2학년)",1535,마더텅,2018-03-01
6,2497,생명이란 무엇인가,1196,까치(까치글방),2021-01-01
7,1137,잃어버린 아이 이야기,534,한길사,2017-12-01
8,1059,데미안,489,민음사,2000-12-01
9,3345,초등영어 받아쓰기·듣기 10회 모의고사 초등 6학년 1,1568,마더텅,2019-04-01
10,1092,동주와 빈센트,503,저녁달고양이,2019-09-01


In [76]:
DATABASE_URL = "oracle+cx_oracle://sys:multisqld@{}:1521/xe?mode=SYSDBA".format(domain_name)
engine = create_engine(DATABASE_URL)

In [77]:
%%SQL
ALTER SESSION SET CONTAINER = XEPDB1;

'변경된 행의 수: 0'

- TEST에게서 lib.book1에 대한 SELECT 권한을 회수합니다. 

In [78]:
%%SQL

REVOKE SELECT ON lib.book1 FROM TEST;

'변경된 행의 수: 0'

In [79]:
DATABASE_URL = "oracle+cx_oracle://test:test123@{}:1521/?service_name=XEPDB1".format(domain_name)
engine = create_engine(DATABASE_URL)

In [80]:
%%SQL

SELECT * FROM lib.book1

ORA-00942: table or view does not exist None


In [81]:
DATABASE_URL = "oracle+cx_oracle://sys:multisqld@{}:1521/xe?mode=SYSDBA".format(domain_name)
engine = create_engine(DATABASE_URL)

In [83]:
%%SQL
ALTER SESSION SET CONTAINER = XEPDB1;

'변경된 행의 수: 0'

In [93]:
%%SQL
SELECT s.sid, s.serial#, s.username, s.status, s.osuser, s.machine
FROM v$session s
WHERE s.username = 'TEST'

,sid,SERIAL#,username,status,osuser,machine


In [91]:
"""
%%SQL

ALTER SYSTEM KILL SESSION '282,60825' IMMEDIATE
"""

'변경된 행의 수: 0'

In [92]:
%%SQL

DROP USER test

'변경된 행의 수: 0'